In [62]:
import pandas as pd
import json
import numpy as np
import yaml

with open('article_publication.json') as f:
    article_publication = json.load(f)
    
df = pd.DataFrame(article_publication)
df = df.fillna(0)
df_normalize = df / df.sum(axis=0)
    
with open('config.yaml') as f:
    biases = pd.Series(yaml.load(f)['publication_bias'])
    
publications = biases[biases > 0].sort_values().index
words = list(list(article_publication.values())[0].keys())

In [63]:
dd = {}
for i in range(df_normalize.shape[0]):
    corr = np.corrcoef(df_normalize.iloc[i], biases)[0][1]
    dd[df_normalize.index[i]] = corr

with open('corr.json', 'w') as f:
    json.dump(dd, f)

In [64]:
with open('corr.json') as f:
    dd = json.load(f)

In [65]:
# words_filtered = df_normalize[(df_normalize.sum(axis=1) > 5000)].dropna().index
# words_filtered = set(words_filtered)

In [77]:
words_filtered = set(df_normalize.dropna().index)

In [80]:
dd_filtered = {k : v for k, v in dd.items() if k in words_filtered}
k_vs = sorted(dd_filtered.items(), key=lambda k_v: k_v[1], reverse=True)

In [96]:
keyword_max_dict = {keyword: max(df_normalize[publication][keyword] for publication in publications)
                    for keyword in words_filtered}

In [99]:
using_keywords = [k for k, v in k_vs]
filtered_keywords = ['1056', '2571']
okay_keywords = []
for k in filtered_keywords + okay_keywords:
    using_keywords.remove(k)
using_keywords = using_keywords[:5] + using_keywords[-5:]

with open('../visualization/d3/data_1.csv', 'w') as f:
    f.write('Media,Bias Level,' + ','.join(using_keywords) + '\n')
    for publication in publications:
        f.write(','.join([publication, str(biases[publication]), *[str(df_normalize[publication][keyword] / keyword_max_dict[keyword]) for keyword in using_keywords]]) + '\n')